In [196]:
import requests
import pandas as pd
import numpy as np

start_date = int(pd.to_datetime('2025-02-01').timestamp())
end_date = int(pd.to_datetime('2025-03-01').timestamp())

response = requests.get(f'https://sensornet.nl/dataserver3/event/collection/nina_events/stream?conditions%5B0%5D%5B%5D=time&conditions%5B0%5D%5B%5D=%3E%3D&conditions%5B0%5D%5B%5D={start_date}&conditions%5B1%5D%5B%5D=time&conditions%5B1%5D%5B%5D=%3C&conditions%5B1%5D%5B%5D={end_date}&conditions%5B2%5D%5B%5D=label&conditions%5B2%5D%5B%5D=in&conditions%5B2%5D%5B2%5D%5B%5D=21&conditions%5B2%5D%5B2%5D%5B%5D=32&conditions%5B2%5D%5B2%5D%5B%5D=33&conditions%5B2%5D%5B2%5D%5B%5D=34&args%5B%5D=aalsmeer&args%5B%5D=schiphol&fields%5B%5D=time&fields%5B%5D=location_short&fields%5B%5D=location_long&fields%5B%5D=duration&fields%5B%5D=SEL&fields%5B%5D=SELd&fields%5B%5D=SELe&fields%5B%5D=SELn&fields%5B%5D=SELden&fields%5B%5D=SEL_dB&fields%5B%5D=lasmax_dB&fields%5B%5D=callsign&fields%5B%5D=type&fields%5B%5D=altitude&fields%5B%5D=distance&fields%5B%5D=winddirection&fields%5B%5D=windspeed&fields%5B%5D=label&fields%5B%5D=hex_s&fields%5B%5D=registration&fields%5B%5D=icao_type&fields%5B%5D=serial&fields%5B%5D=operator&fields%5B%5D=tags')

colnames = pd.DataFrame(response.json()['metadata'])
data = pd.DataFrame(response.json()['rows'])
data.columns = colnames.headers

data['time'] = pd.to_datetime(data['time'], unit = 's')

print(data['time'].min(),data['time'].max())

data.head()

2025-02-01 02:05:41 2025-02-28 22:57:28


headers,id,location_short,location_long,time,SELd,SELe,SELn,SELden,SEL,SEL_dB,...,winddirection,callsign,type,tags,duration,hex_s,registration,icao_type,serial,operator
0,131108581,Aa,Aalsmeerderweg,2025-02-01 04:56:44,0.0,0.0,10814900.0,108149000,10814900,70.340235,...,160.0,TRA915,Boeing 737-800,Kaagbaan06_T,85,4853D2,PH-HXI,B738,62151,Transavia
1,131108591,Aa,Aalsmeerderweg,2025-02-01 05:10:57,0.0,0.0,8570260.0,85702600,8570260,69.329940,...,124.0,TRA705,Boeing 737-800,Kaagbaan06_T,49,484B2A,PH-HSA,B738,34171,
2,131108601,Aa,Aalsmeerderweg,2025-02-01 05:26:21,0.0,0.0,11207700.0,112077000,11207700,70.495153,...,100.0,TRA121,Boeing 737-800,Kaagbaan06_T,43,484506,PH-HZO,B738,34169,
3,131108611,Aa,Aalsmeerderweg,2025-02-01 05:28:39,0.0,0.0,2758770.0,27587700,2758770,64.407154,...,100.0,TFL3KL,None,Kaagbaan06_T,25,485A84,None,None,None,None
4,131108621,Aa,Aalsmeerderweg,2025-02-01 05:41:07,0.0,0.0,9955540.0,99555400,9955540,69.980650,...,75.0,TRA407,Boeing 737-800,Kaagbaan06_T,52,484C5A,PH-HSC,B738,34173,


In [197]:
from opensky_api import OpenSkyApi
import pandas as pd

# Initialize API
api = OpenSkyApi()

# Get states (this returns an object containing the states data)
s = api.get_states()

# Extract the states and convert to a DataFrame
states = s.states  # This is a list of aircraft state objects
data = [vars(state) for state in states]  # Convert to a list of dictionaries

# Create DataFrame
land = pd.DataFrame(data)

# Display the DataFrame
land.head()

,icao24,callsign,origin_country,time_position,last_contact,longitude,latitude,baro_altitude,on_ground,velocity,true_track,vertical_rate,sensors,geo_altitude,squawk,spi,position_source,category
0,4b1816,SWR935L,Switzerland,1.742906e+09,1742905830,21.0337,39.7140,10965.18,False,214.43,321.23,0.33,None,11262.36,6627,False,0,0
1,ab1644,UAL1666,United States,1.742906e+09,1742905831,-76.6089,38.7875,10812.78,False,264.54,37.49,-5.20,None,10698.48,7057,False,0,0
2,4b1819,SWR252B,Switzerland,1.742906e+09,1742905831,17.4058,45.2067,10668.00,False,248.78,118.00,0.00,None,10721.34,3056,False,0,0
3,aa3cbd,N759P,United States,1.742906e+09,1742905781,-81.3926,40.8115,388.62,False,9.16,321.84,-2.60,None,373.38,5550,False,0,0
4,e8027d,LPE2304,Chile,1.742906e+09,1742905545,-77.3606,-12.0387,3215.64,False,158.57,330.02,5.20,None,NaN,None,False,0,0


In [198]:
import requests
import pandas as pd
import plotly.express as px
from io import StringIO
import plotly.graph_objects as go

# De base URL en headers
url = "https://api.schiphol.nl/public-flights/flights?includedelays=false&page={}&sort=%2BscheduleTime"
headers = {
    "app_id": "c93492b2",
    "app_key": "16a5764ed747d28fc0c58196e7322a04",
    'ResourceVersion': 'v4'
}

# Een lege lijst om alle vluchtdata op te slaan
all_flights_data = []

# Loop over de pagina's (0 tot 29, dus 30 pagina's)
for page in range(50):
    # Stel de volledige URL samen met de pagina
    page_url = url.format(page)
    
    # Haal de gegevens op van de API
    response = requests.get(page_url, headers=headers)
    data = response.json()  # Verkrijg de JSON reactie
    
    # Haal de 'flights' lijst op uit de response
    flights_data = data.get('flights', [])
    
    # Voeg de gegevens toe aan de lijst
    all_flights_data.extend(flights_data)

# Normaliseer de vluchtgegevens naar een DataFrame
df = pd.json_normalize(all_flights_data)

url = "https://nl.wikipedia.org/wiki/Vliegvelden_gesorteerd_naar_IATA-code"

# Set the headers with a User-Agent
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'}

# Fetch the HTML content with requests
response = requests.get(url, headers=headers)

# Use StringIO to convert the HTML content to a file-like object
html_content = StringIO(response.text)

# Use pandas to read the table from the HTML content
tables = pd.read_html(html_content)

# Select the table you want (in your case, the correct one based on the structure of the page)
vliegvelden = tables[1]  # Adjust the index based on the correct table

# Zet de lijstkolom om naar een stringkolom door het eerste element te selecteren:
df['route.destinations'] = df['route.destinations'].apply(lambda x: x[0] if isinstance(x, list) and len(x) > 0 else None)

# Verwijder dubbele IATA-codes en behoud alleen de laatste instantie
vliegvelden_clean = vliegvelden.drop_duplicates(subset='IATA', keep='last')

# Maak een mapping van IATA-code naar de bijbehorende luchthavennaam
mapping = vliegvelden_clean.set_index('IATA')['Luchthaven']
mapping2 = vliegvelden_clean.set_index('IATA')['Stad']
mapping3 = vliegvelden_clean.set_index('IATA')['Land']

# Voeg een nieuwe kolom toe aan df door de afkorting te mappen op de juiste luchthavennaam
df['Luchthaven'] = df['route.destinations'].map(mapping)
df['Stad'] = df['route.destinations'].map(mapping2)
df['Land'] = df['route.destinations'].map(mapping3)

df = df[['aircraftRegistration', 'Luchthaven', 'Stad', 'Land', 'flightName']]
stedenlijst= ['Parijs', 'Brussel', 'Antwerpen', 'Praag', 'Londen', 'Hamburg', 'Frankfurt', 'Wenen', 'Luxemburg', 'Milaan', 'Venetië', 'Berlijn', 'München', 'Luxemburg-stad', 'Zurich', 'Marseille', 'Nice', 'Kopenhagen', 'Geneve', 'Luxemburg-Stad']
StedenHackaton= df['Stad'].isin(stedenlijst)
filtered_df = df.loc[StedenHackaton,'Stad']

In [309]:
import pandas as pd

livevlucht = pd.read_csv("flights_today_master.csv", delimiter=',')
livevlucht.head(50)

,Time,Latitude,Longitude,Course,Speed_kts,Speed_mph,Altitude_feet,ClimbRate,ReportingFacility,FlightType,FlightNumber,ScrapeTime
0,Mon 07:13:52 AM,51.5050,0.0604,→ 95°,136.0,157.0,NaN,NaN,FlightAware ADS-B (LCY / EGLC),Arrivals,CFE8489,2025-03-24 12:58:24
1,Mon 07:14:08 AM,51.5043,0.0751,→ 92°,125.0,144.0,950,"3,047",FlightAware ADS-B (LCY / EGLC),Arrivals,CFE8489,2025-03-24 12:58:24
2,Mon 07:14:24 AM,51.5041,0.0909,→ 92°,131.0,151.0,"1,575","1,641",FlightAware ADS-B (LCY / EGLC),Arrivals,CFE8489,2025-03-24 12:58:24
3,Mon 07:14:40 AM,51.5055,0.1075,→ 71°,151.0,174.0,"1,825","1,078",FlightAware ADS-B (SEN / EGMC),Arrivals,CFE8489,2025-03-24 12:58:24
4,Mon 07:14:56 AM,51.5104,0.1241,↗ 53°,169.0,194.0,"2,150","1,313",FlightAware ADS-B (EGML),Arrivals,CFE8489,2025-03-24 12:58:24
5,Mon 07:15:12 AM,51.5207,0.1376,↗ 30°,186.0,214.0,"2,525","1,547",FlightAware ADS-B (LCY / EGLC),Arrivals,CFE8489,2025-03-24 12:58:24
6,Mon 07:15:28 AM,51.5341,0.1485,↗ 26°,205.0,236.0,"2,975","2,045",FlightAware ADS-B (LHR / EGLL),Arrivals,CFE8489,2025-03-24 12:58:24
7,Mon 07:15:45 AM,51.5485,0.1598,↗ 30°,208.0,239.0,"3,650","1,591",FlightAware ADS-B (EGTR),Arrivals,CFE8489,2025-03-24 12:58:24
8,Mon 07:16:01 AM,51.5589,0.1776,↗ 61°,206.0,237.0,"3,850",353,FlightAware ADS-B (EGWE),Arrivals,CFE8489,2025-03-24 12:58:24
9,Mon 07:16:19 AM,51.5687,0.1994,↗ 37°,211.0,243.0,"3,850","1,278",FlightAware ADS-B (EGTR),Arrivals,CFE8489,2025-03-24 12:58:24


In [ ]:
import folium
import pandas as pd

# De lijst met vluchtcodes per stad
london = ['BAW441','KLM1016','KLM1016','BAW444','BAW438','BAW428']
parijs = ['TAY9MS','AFR1240','HOP1740A','FR1140A','FR1640','HOP1740']
hamburg = ['KLM1758','KLM1754','KLM1750','KLM1752']
berlijn = ['KLM1784','NJE622F','KLM1782','KLM1770','EZY5281','EZY5285']
munchen = ['DLH2304', 'KLM1854']
frankfurt = ['KLM1818', 'DLH992']
brussel = ['KLM1706', 'KLM1700', 'KLM1702']
luxemburg = ['GER1714', 'KLM1708']
nice = ['KLM1476', 'KLM1472', 'TRA5588', 'KLM1478']
marseille = ['KLM1470', 'KLM1464', 'KLM1466']
venetie = ['EZY7816', 'KLM1628', 'KLM1634']
milaan = ['KLM1614', 'EZY7830', 'ITY112', 'KLM1612', 'EZY7830']
geneve = ['KLM1938', 'KLM1928', 'KLM1932', 'EZY1517', 'KLM1936']
zurich = ['SWR728', 'KLM1920', 'OAW734', 'KLM1926']
wenen = ['AUA371,', 'KLM1900','AUA377','AUA375','AUA371']
praag = ['KLM1354', 'EZY7928', 'KLM1360','KLM1358','EZY7928']
kopenhagen = ['SAS2551', 'SAS549', 'EZY7938', 'KLM1266', 'KLM1270']

# Gewenste vluchtcodes
gewenste_vluchtcodes = london + parijs + hamburg + berlijn + munchen + frankfurt + luxemburg + nice + marseille + venetie + milaan + geneve + zurich + wenen + praag + kopenhagen

# noord =['BAW441', 'TAY9MS', 'KLM1928', 'KLM1464', 'KLM1708', 'AFR1240', 'KLM1612', 'BAW428', 'KLM1612', 'KLM1472', 'KLM1266', 'KLM1770', 'AUA371', 'AUA371', 'KLM1750', 'TAY9MS', 'KLM1928', 'KLM1900']

# Definieer een dictionary voor kleuren per stad
stad_kleuren = {
    'london': 'blue',
    'parijs': 'red',
    'hamburg': 'green',
    'berlijn': 'purple',
    'munchen': 'orange',
    'frankfurt': 'pink',
    'brussel': 'yellow',
    'luxemburg': 'brown',
    'nice': 'lightblue',
    'marseille': 'darkgreen',
    'venetie': 'lightred',
    'milaan': 'darkblue',
    'geneve': 'lightgreen',
    'zurich': 'darkviolet',
    'wenen': 'beige',
    'praag': 'lightgray',
    'kopenhagen': 'gold'
}

# Filter de unieke vluchtcodes die overeenkomen met de lijst van gewenste vluchtcodes
unieke_vluchtcodes = livevlucht[livevlucht['FlightNumber'].isin(gewenste_vluchtcodes)]['FlightNumber'].unique()

# Maak een folium-kaart
m = folium.Map(location=[52.308916, 4.769637], zoom_start=12)

# Itereer over elke unieke vluchtcode
for vluchtcode in unieke_vluchtcodes:
    # Filter de data voor de specifieke vlucht
    vlucht_data = livevlucht[livevlucht['FlightNumber'] == vluchtcode]
    
    # Haal de coördinaten op
    coordinates = vlucht_data[['Latitude', 'Longitude']].values.tolist()

    # Zoek de stad op basis van de vluchtcode
    if vluchtcode in london:
        stad = 'london'
    elif vluchtcode in parijs:
        stad = 'parijs'
    elif vluchtcode in hamburg:
        stad = 'hamburg'
    elif vluchtcode in berlijn:
        stad = 'berlijn'
    elif vluchtcode in munchen:
        stad = 'munchen'
    elif vluchtcode in frankfurt:
        stad = 'frankfurt'
    elif vluchtcode in brussel:
        stad = 'brussel'
    elif vluchtcode in luxemburg:
        stad = 'luxemburg'
    elif vluchtcode in nice:
        stad = 'nice'
    elif vluchtcode in marseille:
        stad = 'marseille'
    elif vluchtcode in venetie:
        stad = 'venetie'
    elif vluchtcode in milaan:
        stad = 'milaan'
    elif vluchtcode in geneve:
        stad = 'geneve'
    elif vluchtcode in zurich:
        stad = 'zurich'
    elif vluchtcode in wenen:
        stad = 'wenen'
    elif vluchtcode in praag:
        stad = 'praag'
    elif vluchtcode in kopenhagen:
        stad = 'kopenhagen'
    
    # Krijg de kleur voor deze stad
    kleur = stad_kleuren.get(stad, 'gray')  # Gebruik 'gray' als fallback kleur
    
    # Voeg een lijn toe voor het vluchtpad met een pop-up
    folium.PolyLine(
        coordinates, 
        weight=3, 
        opacity=1, 
        color=kleur,  # Gebruik de kleur per stad
        popup=f"Flight: {vluchtcode}<br>City: {stad.capitalize()}"  # Pop-up met de vluchtcode
    ).add_to(m)

# Indien je de kaart hier wilt weergeven:
m


In [ ]:
import folium
import pandas as pd

# De lijst met vluchtcodes per stad
london = ['BAW441','KLM1016','KLM1016','BAW444','BAW438','BAW428']
parijs = ['TAY9MS','AFR1240','HOP1740A','FR1140A','FR1640','HOP1740']
hamburg = ['KLM1758','KLM1754','KLM1750','KLM1752']
berlijn = ['KLM1784','NJE622F','KLM1782','KLM1770','EZY5281','EZY5285']
munchen = ['DLH2304', 'KLM1854']
frankfurt = ['KLM1818', 'DLH992']
brussel = ['KLM1706', 'KLM1700', 'KLM1702']
luxemburg = ['GER1714', 'KLM1708']
nice = ['KLM1476', 'KLM1472', 'TRA5588', 'KLM1478']
marseille = ['KLM1470', 'KLM1464', 'KLM1466']
venetie = ['EZY7816', 'KLM1628', 'KLM1634']
milaan = ['KLM1614', 'EZY7830', 'ITY112', 'KLM1612', 'EZY7830']
geneve = ['KLM1938', 'KLM1928', 'KLM1932', 'EZY1517', 'KLM1936']
zurich = ['SWR728', 'KLM1920', 'OAW734', 'KLM1926']
wenen = ['AUA371,', 'KLM1900','AUA377','AUA375','AUA371']
praag = ['KLM1354', 'EZY7928', 'KLM1360','KLM1358','EZY7928']
kopenhagen = ['SAS2551', 'SAS549', 'EZY7938', 'KLM1266', 'KLM1270']

# Gewenste vluchtcodes
gewenste_vluchtcodes = noord =['BAW441', 'TAY9MS', 'KLM1928', 'KLM1464', 'KLM1708', 'AFR1240', 'KLM1612', 'BAW428', 'KLM1612', 'KLM1472', 'KLM1266', 'KLM1770', 'AUA371', 'AUA371', 'KLM1750', 'TAY9MS', 'KLM1928', 'KLM1900']

#london + parijs + hamburg + berlijn + munchen + frankfurt + luxemburg + nice + marseille + venetie + milaan + geneve + zurich + wenen + praag + kopenhagen


# Definieer een dictionary voor kleuren per stad
stad_kleuren = {
    'london': 'blue',
    'parijs': 'red',
    'hamburg': 'green',
    'berlijn': 'purple',
    'munchen': 'orange',
    'frankfurt': 'pink',
    'brussel': 'yellow',
    'luxemburg': 'brown',
    'nice': 'lightblue',
    'marseille': 'darkgreen',
    'venetie': 'lightred',
    'milaan': 'darkblue',
    'geneve': 'lightgreen',
    'zurich': 'darkviolet',
    'wenen': 'beige',
    'praag': 'lightgray',
    'kopenhagen': 'gold'
}

# Filter de unieke vluchtcodes die overeenkomen met de lijst van gewenste vluchtcodes
unieke_vluchtcodes = livevlucht[livevlucht['FlightNumber'].isin(gewenste_vluchtcodes)]['FlightNumber'].unique()

# Maak een folium-kaart
m = folium.Map(location=[52.308916, 4.769637], zoom_start=12)

# Itereer over elke unieke vluchtcode
for vluchtcode in unieke_vluchtcodes:
    # Filter de data voor de specifieke vlucht
    vlucht_data = livevlucht[livevlucht['FlightNumber'] == vluchtcode]
    
    # Haal de coördinaten op
    coordinates = vlucht_data[['Latitude', 'Longitude']].values.tolist()

    # Zoek de stad op basis van de vluchtcode
    if vluchtcode in london:
        stad = 'london'
    elif vluchtcode in parijs:
        stad = 'parijs'
    elif vluchtcode in hamburg:
        stad = 'hamburg'
    elif vluchtcode in berlijn:
        stad = 'berlijn'
    elif vluchtcode in munchen:
        stad = 'munchen'
    elif vluchtcode in frankfurt:
        stad = 'frankfurt'
    elif vluchtcode in brussel:
        stad = 'brussel'
    elif vluchtcode in luxemburg:
        stad = 'luxemburg'
    elif vluchtcode in nice:
        stad = 'nice'
    elif vluchtcode in marseille:
        stad = 'marseille'
    elif vluchtcode in venetie:
        stad = 'venetie'
    elif vluchtcode in milaan:
        stad = 'milaan'
    elif vluchtcode in geneve:
        stad = 'geneve'
    elif vluchtcode in zurich:
        stad = 'zurich'
    elif vluchtcode in wenen:
        stad = 'wenen'
    elif vluchtcode in praag:
        stad = 'praag'
    elif vluchtcode in kopenhagen:
        stad = 'kopenhagen'
    
    # Krijg de kleur voor deze stad
    kleur = stad_kleuren.get(stad, 'gray')  # Gebruik 'gray' als fallback kleur
    
    # Voeg een lijn toe voor het vluchtpad met een pop-up
    folium.PolyLine(
        coordinates, 
        weight=3, 
        opacity=1, 
        color=kleur,  # Gebruik de kleur per stad
        popup=f"Flight: {vluchtcode}<br>City: {stad.capitalize()}"  # Pop-up met de vluchtcode
    ).add_to(m)

# Indien je de kaart hier wilt weergeven:
m